# Normal Day Analysis

In [2]:
import pandas as pd
import numpy as np
import ipaddress
import dns.resolver
import dns.reversename
import pygeoip
import matplotlib.pyplot as plt 
# Step 5: Adjust the pandas display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 240)
pd.set_option('display.colheader_justify', 'left')
pd.set_option('display.expand_frame_repr', False)


In [3]:
def ts_to_hour(timestamp):
        # Convert 1/100th of seconds to total seconds
    total_seconds = timestamp / 100
    
    # Calculate hours, minutes, and seconds
    hours = int(total_seconds // 3600)
    remaining_seconds_after_hours = total_seconds % 3600
    minutes = int(remaining_seconds_after_hours // 60)
    seconds = remaining_seconds_after_hours % 60
    
    # Format seconds to include fractional part
    hms = f"{hours:02}"
    return hms


In [4]:
datafile='data7.parquet'

In [5]:
gi=pygeoip.GeoIP('./GeoIP.dat')
gi2=pygeoip.GeoIP('./GeoIPASNum.dat')
addr='193.136.73.21'
cc=gi.country_code_by_addr(addr)
org=gi2.org_by_addr(addr)
print(cc,org)

PT AS1930 Fundacao para a Ciencia e a Tecnologia, I.P.


In [6]:
data=pd.read_parquet(datafile)

In [7]:
data.head()

,timestamp,src_ip,dst_ip,proto,port,up_bytes,down_bytes
index,,,,,,,
709945,1232147,192.168.107.196,192.168.107.234,udp,53,157,224
709946,1232151,192.168.107.196,192.168.107.234,udp,53,157,227
709947,1232156,192.168.107.196,192.168.107.234,udp,53,154,219
709948,1232265,192.168.107.196,157.240.212.35,tcp,443,13896,113280
709949,1232398,192.168.107.196,157.240.212.35,tcp,443,45276,419472


In [8]:
NET=ipaddress.IPv4Network('192.168.107.0/24')
private_dst_normal = data[data.apply(lambda x: ipaddress.IPv4Address(x['dst_ip']) in NET,axis=1)]

private_dst_normal.head()


,timestamp,src_ip,dst_ip,proto,port,up_bytes,down_bytes
index,,,,,,,
709945,1232147,192.168.107.196,192.168.107.234,udp,53,157,224
709946,1232151,192.168.107.196,192.168.107.234,udp,53,157,227
709947,1232156,192.168.107.196,192.168.107.234,udp,53,154,219
709954,1232769,192.168.107.196,192.168.107.226,tcp,443,13661,134605
709967,1234167,192.168.107.196,192.168.107.226,tcp,443,5129,39416


In [9]:
private_dst_normal.groupby("src_ip")["dst_ip"].agg(set)

src_ip
192.168.107.100    {192.168.107.222, 192.168.107.238, 192.168.107...
192.168.107.101    {192.168.107.222, 192.168.107.238, 192.168.107...
192.168.107.102    {192.168.107.222, 192.168.107.238, 192.168.107...
192.168.107.103    {192.168.107.222, 192.168.107.238, 192.168.107...
192.168.107.104    {192.168.107.222, 192.168.107.238, 192.168.107...
                                         ...                        
192.168.107.95     {192.168.107.222, 192.168.107.238, 192.168.107...
192.168.107.96     {192.168.107.222, 192.168.107.238, 192.168.107...
192.168.107.97     {192.168.107.222, 192.168.107.238, 192.168.107...
192.168.107.98     {192.168.107.222, 192.168.107.238, 192.168.107...
192.168.107.99     {192.168.107.222, 192.168.107.238, 192.168.107...
Name: dst_ip, Length: 199, dtype: object

### Comunicacoes com ip's internos
"{'192.168.107.226', '192.168.107.223', '192.168.107.234', '192.168.107.227', '192.168.107.222', '192.168.107.238'}"

In [10]:
grp_private_normal = private_dst_normal.groupby(["proto","port"])['dst_ip'].apply(list)

In [39]:
data.groupby("src_ip")["dst_ip"].count().reset_index().sort_values(by="dst_ip", ascending=False)

,src_ip,dst_ip
44,192.168.107.140,12666
42,192.168.107.139,11758
150,192.168.107.51,11739
145,192.168.107.46,10673
169,192.168.107.70,10622
...,...,...
8,192.168.107.108,892
180,192.168.107.81,597
196,192.168.107.97,570
121,192.168.107.22,499


In [53]:
data_ratios = data.copy()

In [54]:
data_ratios.loc[:,"ratio"] = data_ratios["up_bytes"] / data_ratios["down_bytes"]

In [64]:
data.groupby("src_ip")["down_bytes"].min().mean()

187.3819095477387

## Max Up bytes and down bytes on average
66764 up bytes

3959601 down bytes

## Min Up bytes and down bytes on average

114 up bytes

187 down bytes

## Min Ratio on average

0.155 ratio (up/down)

## Max Ratio on average

0.174 ratio (up/down)


In [130]:
data.groupby("src_ip").agg({
"up_bytes" : ["min","max","mean"],"down_bytes": ["min","max","mean"]
}).reset_index()


src_ip           up_bytes                      down_bytes                       
                     min      max    mean          min        max      mean         
0    192.168.107.100  134      64916  10116.875462  200        2194635  92705.122894
1    192.168.107.101  110      72872  10025.303948  176        3898754  94732.699947
2    192.168.107.102  129      78041   9678.595481  221        2814076  90240.021866
3    192.168.107.103  105      92018   9972.337606  168        4794581  94036.708612
4    192.168.107.104  108      67244   9873.292324  159        3862085  93812.450054
..               ...      ...    ...           ...        ...      ...           ...
194   192.168.107.95  107      68304  10043.766040  151        4577195  94093.268957
195   192.168.107.96  121      67178   9938.511026  196        2871762  88812.157534
196   192.168.107.97  125      35646   9966.408772  277         371164  84183.878947
197   192.168.107.98  113      59741  10031.142893  175        3468603  94293.416155
198   192.168.107.99  139      48841  10080.663830  247        2885880  97887.336170

[199 rows x 7 columns]

### Horas de trabalho

In [14]:
maxt = private_dst_normal.groupby("src_ip")["timestamp"].apply(max).apply(ts_to_hour).reset_index()
mint = private_dst_normal.groupby("src_ip")["timestamp"].apply(min).apply(ts_to_hour).reset_index()


/tmp/ipykernel_6017/2062456772.py:1: FutureWarning: The provided callable <built-in function max> is currently using np.maximum.reduce. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string np.maximum.reduce instead.
  maxt = private_dst_normal.groupby("src_ip")["timestamp"].apply(max).apply(ts_to_hour).reset_index()
/tmp/ipykernel_6017/2062456772.py:2: FutureWarning: The provided callable <built-in function min> is currently using np.minimum.reduce. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string np.minimum.reduce instead.
  mint = private_dst_normal.groupby("src_ip")["timestamp"].apply(min).apply(ts_to_hour).reset_index()


In [15]:
mint.head()

,src_ip,timestamp
0,192.168.107.100,11
1,192.168.107.101,08
2,192.168.107.102,08
3,192.168.107.103,10
4,192.168.107.104,05


In [16]:
maxt.head()

,src_ip,timestamp
0,192.168.107.100,15
1,192.168.107.101,17
2,192.168.107.102,12
3,192.168.107.103,19
4,192.168.107.104,17


In [17]:
ts_hours = pd.merge(mint, maxt, on="src_ip",suffixes=("_mint","_maxt"))

In [18]:
ts_hours.head()

,src_ip,timestamp_mint,timestamp_maxt
0,192.168.107.100,11,15
1,192.168.107.101,08,17
2,192.168.107.102,08,12
3,192.168.107.103,10,19
4,192.168.107.104,05,17


In [80]:
from collections import Counter
def count_letters(letter_list):
    return dict(Counter(letter_list))
def get_top_3_counts(letter_count,n=3):
    # Sort the counts in descending order and get the top 3
    return dict(Counter(letter_count).most_common(n))

ts_hours.groupby("timestamp_mint")["timestamp_maxt"].apply(list).apply(count_letters).apply(get_top_3_counts).reset_index()

,timestamp_mint,timestamp_maxt
0,03,{'17': 1}
1,04,"{'14': 2, '16': 1}"
2,05,"{'17': 3, '15': 3, '21': 2}"
3,06,"{'17': 4, '15': 2, '18': 1}"
4,07,"{'17': 6, '15': 5, '18': 4}"
5,08,"{'14': 5, '18': 5, '19': 5}"
6,09,"{'17': 8, '15': 7, '14': 3}"
7,10,"{'17': 7, '19': 4, '16': 3}"
8,11,"{'16': 4, '15': 3, '18': 3}"
9,12,"{'17': 3, '21': 1}"


### Horas de trabalho mais comuns
| Hora de inicio | hora de fim ({Hora: Contagem}) |
|:----:|:------:|
| 03 |    {'17': 1} |
| 04 |    {'14': 2} |
| 05 |    {'17': 3} |
| 06 |    {'17': 4} |
| 07 |    {'17': 6} |
| 08 |    {'14': 5} |
| 09 |    {'17': 8} |
| 10 |    {'17': 7} |
| 11 |    {'16': 4} |
| 12 |    {'17': 3} |
| 13 |    {'17': 1} |

Hora mais comum e das 9 as 17.

Mas o intervalo mais abrangente e das 7 as 17.

In [66]:

https_servers = set(grp_private_normal[('tcp', 443)])
print()

dns_servers =  set(grp_private_normal[('udp', 53)])
print()

## Servers 
tcp 443 {'192.168.107.223', '192.168.107.238', '192.168.107.222', '192.168.107.226'} https

udp 53 {'192.168.107.227', '192.168.107.234'} dns


In [67]:
grouped = data.groupby(['src_ip','dst_ip']).sum()

# Calculate the ratio
grouped['ratio'] = grouped['up_bytes'] / grouped['down_bytes']
grouped
# Reset the index to make dst_ip a column again
# grouped = grouped.reset_index()

timestamp proto                                               port   up_bytes  down_bytes  ratio   
src_ip          dst_ip                                                                                                             
192.168.107.100 104.16.249.130   94725010  tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   7974  209457    1727986     0.121215
                104.22.48.100    36323031                        tcptcptcptcptcptcptcptcptcp   3987   93381     814578     0.114637
                104.244.42.129  246717524  tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...  21707  512007    4246664     0.120567
                104.26.13.207   100077577  tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   8860  270821    6675043     0.040572
                104.26.14.51     79996025   tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcptcp   7088  205239    1890983     0.108536
...                                   ...                                                ...    ...       ...         ...       ...
192.168.107.99  66.22.229.4      11237553                                          tcptcptcp   1329   23809     267000     0.089172
                66.254.114.41    10868618                                          tcptcptcp   1329   37602     337169     0.111523
                88.157.217.145   66107954  tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   7974  159654    1506733     0.105960
                88.157.217.146  120743878  tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...  14619  379690    4234007     0.089676
                97.107.142.234   30141613                           tcptcptcptcptcptcptcptcp   3544   71568     677703     0.105604

[27120 rows x 6 columns]

In [68]:
mean_ratios = grouped.groupby('src_ip')['ratio'].mean().reset_index()

In [81]:
server_traffic = data[data.apply(lambda x: x['dst_ip'] in dns_servers or x['dst_ip'] in https_servers,axis=1)]



In [82]:
server_traffic.head()

,timestamp,src_ip,dst_ip,proto,port,up_bytes,down_bytes
index,,,,,,,
709945,1232147,192.168.107.196,192.168.107.234,udp,53,157,224
709946,1232151,192.168.107.196,192.168.107.234,udp,53,157,227
709947,1232156,192.168.107.196,192.168.107.234,udp,53,154,219
709954,1232769,192.168.107.196,192.168.107.226,tcp,443,13661,134605
709967,1234167,192.168.107.196,192.168.107.226,tcp,443,5129,39416


#### src_ip connections to servers both dns or tcp

In [88]:
server_traffic_dns = data[data.apply(lambda x: x['dst_ip'] in dns_servers,axis=1)]

server_traffic_http =  data[data.apply(lambda x: x['dst_ip'] in https_servers,axis=1)]

In [102]:
server_traffic_dns.groupby("src_ip").sum().reset_index().drop(["dst_ip","proto","port","timestamp"],axis=1).groupby("src_ip").mean()

,up_bytes,down_bytes
src_ip,,
192.168.107.100,53436.0,121293.0
192.168.107.101,140308.0,323678.0
192.168.107.102,39243.0,88961.0
192.168.107.103,134108.0,303115.0
192.168.107.104,200876.0,461519.0
...,...,...
192.168.107.95,162180.0,377470.0
192.168.107.96,151766.0,347889.0
192.168.107.97,15558.0,34960.0


In [103]:
server_traffic_http.groupby("src_ip").sum().reset_index().drop(["dst_ip","proto","port","timestamp"],axis=1).groupby("src_ip").mean()

,up_bytes,down_bytes
src_ip,,
192.168.107.100,4624097.0,43566079.0
192.168.107.101,12764744.0,120998792.0
192.168.107.102,2788673.0,28225949.0
192.168.107.103,10145447.0,90000179.0
192.168.107.104,14831777.0,143440608.0
...,...,...
192.168.107.95,13288065.0,117469401.0
192.168.107.96,11856795.0,108232414.0
192.168.107.97,1184687.0,10151913.0


In [110]:
servers_grouped = server_traffic.groupby(['dst_ip','src_ip']).sum()

# Calculate the ratio
servers_grouped['ratio'] = servers_grouped['up_bytes'] / servers_grouped['down_bytes']
servers_grouped
# Reset the index to make dst_ip a column again
# grouped = grouped.reset_index()
servers_mean_ratios = servers_grouped.groupby('dst_ip')['ratio'].mean().reset_index()

In [111]:
servers_mean_ratios.sort_values(by="ratio",ascending=False)

,dst_ip,ratio
3,192.168.107.227,0.435827
4,192.168.107.234,0.434916
0,192.168.107.222,0.110127
1,192.168.107.223,0.109864
2,192.168.107.226,0.108967
5,192.168.107.238,0.108939


In [112]:
servers_mean_ratios.to_csv("dst_ip_normal.csv")

In [113]:
NET=ipaddress.IPv4Network('192.168.107.0/24')
private_dst_internal_normal = data[data.apply(lambda x: ipaddress.IPv4Address(x['dst_ip']) in NET  and  ipaddress.IPv4Address(x['src_ip']) in NET  ,axis=1)]

private_dst_internal_normal.head()

,timestamp,src_ip,dst_ip,proto,port,up_bytes,down_bytes
index,,,,,,,
709945,1232147,192.168.107.196,192.168.107.234,udp,53,157,224
709946,1232151,192.168.107.196,192.168.107.234,udp,53,157,227
709947,1232156,192.168.107.196,192.168.107.234,udp,53,154,219
709954,1232769,192.168.107.196,192.168.107.226,tcp,443,13661,134605
709967,1234167,192.168.107.196,192.168.107.226,tcp,443,5129,39416


In [114]:
private_dst_internal_normal.groupby("src_ip")['dst_ip'].agg(set).reset_index().to_csv("normal_coms_internal.csv")

print(private_dst_internal_normal.groupby("src_ip")['dst_ip'].agg(list).reset_index()[:]['src_ip'][0])
print(set(private_dst_internal_normal.groupby("src_ip")['dst_ip'].agg(list).reset_index()[:]['dst_ip'][0]))

192.168.107.100
{'192.168.107.222', '192.168.107.238', '192.168.107.226', '192.168.107.227', '192.168.107.223', '192.168.107.234'}


In [106]:
exterior = data[data.apply(lambda x: ipaddress.IPv4Address(x['dst_ip']) not in NET,axis=1)]
exterior.loc[:,'ratio'] = exterior['up_bytes']/exterior['down_bytes']

/tmp/ipykernel_6017/4032262708.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exterior.loc[:,'ratio'] = exterior['up_bytes']/exterior['down_bytes']


In [107]:
grp_exterior_normal = exterior.groupby(["proto","port"])['dst_ip'].apply(list)

https_servers = set(grp_exterior_normal[('tcp', 443)])
print(https_servers)

ips_cc = {addr : gi.country_code_by_addr(addr) for addr in https_servers}
ips_cc_filtered = {k:v for k,v in ips_cc.items() if k not in ['US','GB','UK']}

# dns_servers =  set(grp_exterior_normal[('udp', 53)])
# print(dns_servers)

{'40.66.0.201', '66.246.138.175', '176.32.124.226', '40.79.60.58', '52.125.134.22', '40.79.90.205', '52.126.224.15', '170.187.140.138', '52.123.192.85', '13.34.7.79', '52.95.22.207', '52.126.237.135', '192.241.169.114', '151.236.222.112', '40.77.200.181', '52.180.185.83', '52.219.233.240', '20.95.240.218', '52.95.230.178', '68.220.86.141', '157.55.214.47', '104.47.202.136', '143.198.245.154', '40.90.71.86', '52.113.128.240', '52.113.204.140', '40.93.212.114', '140.204.58.241', '147.154.117.131', '52.144.193.103', '157.56.31.65', '69.164.203.235', '150.222.178.17', '198.199.105.91', '130.35.202.148', '138.2.244.202', '178.79.147.49', '198.211.101.60', '143.42.189.15', '13.107.253.140', '51.141.137.150', '150.222.214.222', '40.75.39.3', '52.103.166.196', '65.55.145.243', '15.230.255.127', '209.17.60.86', '150.222.110.18', '199.30.28.152', '51.141.138.168', '209.17.61.47', '199.30.27.19', '74.119.79.73', '52.115.92.23', '65.55.252.128', '40.77.201.89', '52.144.197.165', '52.93.62.53', '40

In [108]:
exterior.groupby(['src_ip','dst_ip'])["ratio"].mean().reset_index().sort_values(ascending=False, by='ratio').to_csv("dst_ip_ratio_normal.csv")

In [118]:
print("Numero de paises contactados", len(set(ips_cc.values())))

Numero de paises contactados 35
